In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

In [78]:
# Load 2016 Census Data
df_2016 = pd.read_csv('98-401-X2016045_English_CSV_data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [79]:
# Load ML model
reg = pickle.load(open("census_crime_model.sav", 'rb'))

In [119]:
reg.coef_

array([-2.45022406e-02, -3.24680811e+02, -3.04122581e-01, -3.92807982e-01,
        6.13627017e-01, -2.38926981e+01,  2.46544092e+01,  2.35764958e+01,
        1.78797250e-01, -6.09535253e+01])

# Load and Process Montreal Census Data

In [81]:
data = {
    'YEAR': [],
    'GEO_CODE': [],
    'GEO_NAME': [],
    'POPULATION_DENSITY': [],
    'POPULATION_PERCENTAGE_CHANGE': [],
    'POP_TOTAL': [],
    'POP_MALE': [],
    'POP_FEMALE': [],
    'CHILDREN_TOTAL': [],
    'CHILDREN_MALE': [],
    'CHILDREN_FEMALE': [],
    'YOUTH_TOTAL': [],
    'YOUTH_MALE': [],
    'YOUTH_FEMALE': [],
    'ADULT_TOTAL': [],
    'ADULT_MALE': [],
    'ADULT_FEMALE': [],
    'SENIOR_TOTAL': [],
    'SENIOR_MALE': [],
    'SENIOR_FEMALE': [],
    'HOUSEHOLD_INCOME': [],
    'EMPLOYMENT_RATE': [],
    'UNEMPLOYMENT_RATE': []
    }

# Create DataFrame  
df_census = pd.DataFrame(data)  

In [82]:
def load_census_2016(Geocodes):
    
    data = {
        'YEAR': [],
        'GEO_CODE': [],
        'GEO_NAME': [],
        'POPULATION_DENSITY': [],
        'POPULATION_PERCENTAGE_CHANGE': [],
        'POP_TOTAL': [],
        'POP_MALE': [],
        'POP_FEMALE': [],
        'CHILDREN_TOTAL': [],
        'CHILDREN_MALE': [],
        'CHILDREN_FEMALE': [],
        'YOUTH_TOTAL': [],
        'YOUTH_MALE': [],
        'YOUTH_FEMALE': [],
        'ADULT_TOTAL': [],
        'ADULT_MALE': [],
        'ADULT_FEMALE': [],
        'SENIOR_TOTAL': [],
        'SENIOR_MALE': [],
        'SENIOR_FEMALE': [],
        'HOUSEHOLD_INCOME': [],
        'EMPLOYMENT_RATE': [],
        'UNEMPLOYMENT_RATE': []
        }

    # Create DataFrame  
    df = pd.DataFrame(data) 
    year = "2016"

    for geocode in Geocodes:
        df_tmpt = df_2016[df_2016['GEO_CODE (POR)'] == geocode]
        geo_name = df_tmpt['GEO_NAME'].iloc[0]

        # Population density per square kilometre
        population_density = float(df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Population density per square kilometre"]["Dim: Sex (3): Member ID: [1]: Total - Sex"])

        # Population percentage change, 2011 to 2016
        population_percentage_change = float(df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Population percentage change, 2011 to 2016"]["Dim: Sex (3): Member ID: [1]: Total - Sex"])

        # age group:
        # Children (00-14 years)
        # Youth (15-24 years)
        # Adults (25-64 years)
        # Seniors (65 years and over)

        total_population = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Total - Age groups and average age of the population - 100% data"]
        total_population_all = int(total_population['Dim: Sex (3): Member ID: [1]: Total - Sex'])
        total_population_male = int(total_population['Dim: Sex (3): Member ID: [2]: Male'])
        total_population_female = int(total_population['Dim: Sex (3): Member ID: [3]: Female'])

        age_children = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "0 to 14 years"].iloc[0]
        total_num_children = int(age_children['Dim: Sex (3): Member ID: [1]: Total - Sex'])
        male_num_children = int(age_children['Dim: Sex (3): Member ID: [2]: Male'])
        female_num_children = int(age_children['Dim: Sex (3): Member ID: [3]: Female'])

        age_15_64 = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "15 to 64 years"].iloc[0]
        age_15_19 = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "15 to 19 years"].iloc[0]
        age_19_24 = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "20 to 24 years"].iloc[0]

        total_num_youth = int(age_15_19['Dim: Sex (3): Member ID: [1]: Total - Sex']) + int(age_19_24['Dim: Sex (3): Member ID: [1]: Total - Sex'])
        male_num_youth = int(age_15_19['Dim: Sex (3): Member ID: [2]: Male']) + int(age_19_24['Dim: Sex (3): Member ID: [2]: Male'])
        female_num_youth = int(age_15_19['Dim: Sex (3): Member ID: [3]: Female']) + int(age_19_24['Dim: Sex (3): Member ID: [3]: Female'])

        total_num_adult = int(age_15_64['Dim: Sex (3): Member ID: [1]: Total - Sex']) - total_num_youth
        male_num_adult = int(age_15_64['Dim: Sex (3): Member ID: [2]: Male']) - male_num_youth
        female_num_adult = int(age_15_64['Dim: Sex (3): Member ID: [3]: Female']) - female_num_youth

        age_senior = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "65 years and over"].iloc[0]
        total_num_senior = int(age_senior['Dim: Sex (3): Member ID: [1]: Total - Sex'])
        male_num_senior = int(age_senior['Dim: Sex (3): Member ID: [2]: Male'])
        female_num_senior = int(age_senior['Dim: Sex (3): Member ID: [3]: Female'])

        household_income = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Total - Household total income groups in 2015 for private households - 100% data"]
        household_income_total = float(household_income['Dim: Sex (3): Member ID: [1]: Total - Sex'])

        employment_rate = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Employment rate"]
        employment_rate_total = float(employment_rate['Dim: Sex (3): Member ID: [1]: Total - Sex'])

        unemployment_rate = df_tmpt[df_tmpt["DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)"] == "Unemployment rate"]
        unemployment_rate_total = float(unemployment_rate['Dim: Sex (3): Member ID: [1]: Total - Sex'])

        new_data = {
              'YEAR': year,
              'GEO_CODE': geocode,
              'GEO_NAME': geo_name,
              'POPULATION_DENSITY': population_density,
              'POPULATION_PERCENTAGE_CHANGE': population_percentage_change,
              'POP_TOTAL': total_population_all,
              'POP_MALE': total_population_male,
              'POP_FEMALE': total_population_female,
              'CHILDREN_TOTAL': total_num_children,
              'CHILDREN_MALE': male_num_children,
              'CHILDREN_FEMALE': female_num_children,
              'YOUTH_TOTAL': total_num_youth,
              'YOUTH_MALE': male_num_youth,
              'YOUTH_FEMALE': female_num_youth,
              'ADULT_TOTAL': total_num_adult,
              'ADULT_MALE': male_num_adult,
              'ADULT_FEMALE': female_num_adult,
              'SENIOR_TOTAL': total_num_senior,
              'SENIOR_MALE': male_num_senior,
              'SENIOR_FEMALE': female_num_senior,
              'HOUSEHOLD_INCOME': household_income_total,
              'EMPLOYMENT_RATE': employment_rate_total,
              'UNEMPLOYMENT_RATE': unemployment_rate_total
              }

        df = df.append(new_data, ignore_index=True)
    
    return df

In [93]:
geocodes = [24052, 24053, 24055, 24069]
df_census = load_census_2016(geocodes)

In [94]:
df_census.head()

,YEAR,GEO_CODE,GEO_NAME,POPULATION_DENSITY,POPULATION_PERCENTAGE_CHANGE,POP_TOTAL,POP_MALE,POP_FEMALE,CHILDREN_TOTAL,CHILDREN_MALE,...,YOUTH_FEMALE,ADULT_TOTAL,ADULT_MALE,ADULT_FEMALE,SENIOR_TOTAL,SENIOR_MALE,SENIOR_FEMALE,HOUSEHOLD_INCOME,EMPLOYMENT_RATE,UNEMPLOYMENT_RATE
0,2016,24052.0,Mont-Royal,4583.3,2.0,103320.0,48690.0,54630.0,18905.0,9770.0,...,6395.0,51190.0,23805.0,27390.0,20030.0,8315.0,11715.0,40765.0,56.4,8.4
1,2016,24053.0,Notre-Dame-de-Grâce--Westmount,6544.1,0.5,104975.0,49535.0,55440.0,16450.0,8285.0,...,6885.0,56150.0,26820.0,29335.0,18880.0,7830.0,11050.0,47410.0,58.4,8.6
2,2016,24055.0,Papineau,11179.4,1.6,110750.0,55475.0,55275.0,17000.0,8760.0,...,6510.0,65650.0,33640.0,32015.0,14995.0,6480.0,8515.0,52285.0,58.6,10.0
3,2016,24069.0,Saint-Léonard--Saint-Michel,5592.8,2.3,113210.0,53800.0,59410.0,23100.0,11560.0,...,6415.0,57175.0,27535.0,29655.0,20310.0,8505.0,11800.0,44315.0,51.6,12.3


# Load and Procress the Montreal Crime Data

In [95]:
# Load Montreal Crime Data
# https://www.kaggle.com/datasets/stevieknox/montreal-crime-data
df = pd.read_csv('Montreal Crime Data.csv')

In [96]:
df_montreal = df[df['city'] == "MONTREAL"]
df_montreal.head()

,Unnamed: 0,category,date,postal_code,city,neighbourhood,year,count,longitude,latitude
0,0,Motor vehicle theft,2018-09-13,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567
1,1,Motor vehicle theft,2018-04-30,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567
2,2,Home Invasion,2018-01-10,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569
3,3,Mischief,2018-11-12,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569
4,4,Mischief,2018-08-15,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569


In [97]:
df_montreal['year'].max()

2021

In [98]:
df_montreal['year'].min()

2015

In [99]:
df_montreal['neighbourhood'].unique()

array(['Saint-Michel', 'Outremont', 'Montréal-Nord', 'Centre-Sud',
       'Mercier', 'Petite-Patrie', 'Old Montreal', 'Villeray', 'Rosemont',
       'Ahuntsic', 'Parc-Extension', 'Cartierville', 'Côte Saint-Luc',
       'Hochelaga', 'Saint-Léonard', 'Plateau Mont-Royal', 'Anjou',
       'Notre-Dame-de-Grâce', 'Maisonneuve', 'Westmount',
       'Rivière-des-Prairies',
       'Griffintown(Includes Île Notre-Dame & Île Sainte-Hélène)(Université de Montréal)',
       'Saint-Michel,East', 'Ville Émard', 'Downtown Montreal',
       'Saint-Henri', 'Hampstead / Côte Saint-Luc', 'Saint-Laurent',
       'Pointe-Saint-Charles', 'Montreal East', 'Pointe-aux-Trembles',
       'Côte-des-Neiges', 'Mount Royal', 'Petite-Bourgogne'], dtype=object)

In [90]:
df_montreal.groupby('neighbourhood')['category'].count().sort_values(ascending=False).reset_index(name='COUNT')

,neighbourhood,COUNT
0,Plateau Mont-Royal,10844
1,Centre-Sud,7849
2,Downtown Montreal,7565
3,Mercier,6250
4,Rosemont,5651
5,Ahuntsic,5575
6,Petite-Patrie,5327
7,Villeray,4649
8,Côte-des-Neiges,4273
9,Hochelaga,4207


In [107]:
district_montreal_test = {
    "Côte Saint-Luc": "Mont-Royal",
    "Hampstead / Côte Saint-Luc": "Mont-Royal",
    "Mount Royal": "Mont-Royal",
    "Westmount": "Notre-Dame-de-Grâce--Westmount",
    "Notre-Dame-de-Grâce": "Notre-Dame-de-Grâce--Westmount",
    "Villeray": "Papineau",
    "Parc-Extension": "Papineau",
    "Saint-Léonard": "Saint-Léonard--Saint-Michel",
    "Saint-Michel": "Saint-Léonard--Saint-Michel",
    "Saint-Michel,East": "Saint-Léonard--Saint-Michel"
}

neighbourhoods = ["Côte Saint-Luc", "Hampstead / Côte Saint-Luc", 
          "Mount Royal", "Westmount", "Notre-Dame-de-Grâce",
          "Villeray", "Parc-Extension", "Saint-Léonard", 
          "Saint-Michel", "Saint-Michel,East"
          ]

def in_select_neighbourhood (neighbourhood):
    return neighbourhood in neighbourhoods

def neighbourhood_to_district (neighbourhood):
    return district_montreal_test[neighbourhood]

def average_count (count, year):
    return count / year

In [102]:
df_montreal['in_test'] = df_montreal.apply(lambda row: in_select_neighbourhood(row['neighbourhood']), axis=1)
df_montreal = df_montreal[df_montreal['in_test']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
df_montreal

,Unnamed: 0,category,date,postal_code,city,neighbourhood,year,count,longitude,latitude,in_test
0,0,Motor vehicle theft,2018-09-13,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567,True
1,1,Motor vehicle theft,2018-04-30,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567,True
2,2,Home Invasion,2018-01-10,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True
3,3,Mischief,2018-11-12,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True
4,4,Mischief,2018-08-15,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True
...,...,...,...,...,...,...,...,...,...,...,...
135346,135346,Motor vehicle theft,2021-04-12,H4P 1T6,MONTREAL,Mount Royal,2021,1,-73.648,45.500,True
135347,135347,Theft in / from a motor vehicle,2021-08-16,H4P 1T6,MONTREAL,Mount Royal,2021,1,-73.648,45.500,True
135349,135349,Theft in / from a motor vehicle,2021-03-28,H4P 1T2,MONTREAL,Mount Royal,2021,1,-73.649,45.498,True
135350,135350,Confirmed Theft,2021-04-19,H4P 1T2,MONTREAL,Mount Royal,2021,1,-73.649,45.498,True


In [104]:
df_montreal['GEO_NAME'] = df_montreal.apply(lambda row: neighbourhood_to_district(row['neighbourhood']), axis=1)

In [105]:
df_montreal

,Unnamed: 0,category,date,postal_code,city,neighbourhood,year,count,longitude,latitude,in_test,GEO_NAME
0,0,Motor vehicle theft,2018-09-13,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567,True,Saint-Léonard--Saint-Michel
1,1,Motor vehicle theft,2018-04-30,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567,True,Saint-Léonard--Saint-Michel
2,2,Home Invasion,2018-01-10,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True,Saint-Léonard--Saint-Michel
3,3,Mischief,2018-11-12,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True,Saint-Léonard--Saint-Michel
4,4,Mischief,2018-08-15,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569,True,Saint-Léonard--Saint-Michel
...,...,...,...,...,...,...,...,...,...,...,...,...
135346,135346,Motor vehicle theft,2021-04-12,H4P 1T6,MONTREAL,Mount Royal,2021,1,-73.648,45.500,True,Mont-Royal
135347,135347,Theft in / from a motor vehicle,2021-08-16,H4P 1T6,MONTREAL,Mount Royal,2021,1,-73.648,45.500,True,Mont-Royal
135349,135349,Theft in / from a motor vehicle,2021-03-28,H4P 1T2,MONTREAL,Mount Royal,2021,1,-73.649,45.498,True,Mont-Royal
135350,135350,Confirmed Theft,2021-04-19,H4P 1T2,MONTREAL,Mount Royal,2021,1,-73.649,45.498,True,Mont-Royal


In [110]:
df_crime_group = df_montreal.groupby(['GEO_NAME']).count()['category'].reset_index(name='COUNT')
df_crime_group.head()

,GEO_NAME,COUNT
0,Mont-Royal,2099
1,Notre-Dame-de-Grâce--Westmount,3028
2,Papineau,6869
3,Saint-Léonard--Saint-Michel,4457


In [111]:
df_crime_group['AVG_CRIME_COUNT'] = df_crime_group.apply(lambda row: average_count(row['COUNT'], 7), axis=1)
df_crime_group.head()

,GEO_NAME,COUNT,AVG_CRIME_COUNT
0,Mont-Royal,2099,299.857143
1,Notre-Dame-de-Grâce--Westmount,3028,432.571429
2,Papineau,6869,981.285714
3,Saint-Léonard--Saint-Michel,4457,636.714286


In [113]:
df_result = pd.merge(df_census, df_crime_group, on=['GEO_NAME'])
df_result.head()

,YEAR,GEO_CODE,GEO_NAME,POPULATION_DENSITY,POPULATION_PERCENTAGE_CHANGE,POP_TOTAL,POP_MALE,POP_FEMALE,CHILDREN_TOTAL,CHILDREN_MALE,...,ADULT_MALE,ADULT_FEMALE,SENIOR_TOTAL,SENIOR_MALE,SENIOR_FEMALE,HOUSEHOLD_INCOME,EMPLOYMENT_RATE,UNEMPLOYMENT_RATE,COUNT,AVG_CRIME_COUNT
0,2016,24052.0,Mont-Royal,4583.3,2.0,103320.0,48690.0,54630.0,18905.0,9770.0,...,23805.0,27390.0,20030.0,8315.0,11715.0,40765.0,56.4,8.4,2099,299.857143
1,2016,24053.0,Notre-Dame-de-Grâce--Westmount,6544.1,0.5,104975.0,49535.0,55440.0,16450.0,8285.0,...,26820.0,29335.0,18880.0,7830.0,11050.0,47410.0,58.4,8.6,3028,432.571429
2,2016,24055.0,Papineau,11179.4,1.6,110750.0,55475.0,55275.0,17000.0,8760.0,...,33640.0,32015.0,14995.0,6480.0,8515.0,52285.0,58.6,10.0,6869,981.285714
3,2016,24069.0,Saint-Léonard--Saint-Michel,5592.8,2.3,113210.0,53800.0,59410.0,23100.0,11560.0,...,27535.0,29655.0,20310.0,8505.0,11800.0,44315.0,51.6,12.3,4457,636.714286


In [114]:
y = df_result['AVG_CRIME_COUNT']
X = df_result[['POPULATION_DENSITY', 'POPULATION_PERCENTAGE_CHANGE', 
        'POP_MALE', 'CHILDREN_TOTAL', 'YOUTH_MALE', 
        'ADULT_TOTAL', 'ADULT_MALE', 'ADULT_FEMALE',
        'HOUSEHOLD_INCOME', 'EMPLOYMENT_RATE'
        ]]

In [117]:
reg.score(X, y)

-116.3967207291723

In [120]:
reg.predict(X)

array([-1082.05919314,  2689.29898487,  5399.86794611, -1483.05198948])